In [1]:
import numpy as np

from base import *

In [2]:
def print_estimate(models, prm_names):
    for name in prm_names:
        print(f" {name:<10}", end="")
    print()
    for prm, prm_err in models:
        for reg, reg_err in zip(prm, prm_err):
            for est in reg:
                print(f" {est:<8.3g}  ", end="")
            print()
            for err in reg_err.diagonal()**0.5:
                print(f"({err:<8.3g}) ", end="")
            print()

In [3]:
seed = 179
rng = np.random.default_rng(seed)

In [130]:
n = 1001
d = 5

X = rng.standard_normal((d, n))
b = rng.uniform(0, 1, (1, d))
w = rng.uniform(0, 1, n) * 10

tries = 1000
summ = 0

for _ in range(tries):
    Y_err = rng.standard_normal((1, n))
    Y = b @ X + Y_err
    prm, prm_err = estimate_wls(Y, X, w, get_hc1_err)
    dev = prm - b
    summ += np.abs(dev[0][0]) < (prm_err[0].diagonal()[0])**0.5

print(summ / tries)

0.68


In [135]:
n = 1001
d = 5
r = 10

Z = rng.standard_normal((r, n))
g = rng.uniform(0, 1, (d, r))
b = rng.uniform(0, 1, (1, d))
b_err = rng.uniform(0, 1, (1, d))
w = rng.uniform(0, 1, n) * 10

tries = 1000
summ = 0

for _ in range(tries):
    X_err = rng.standard_normal((d, n))
    X = g @ Z + X_err
    Y_err = rng.standard_normal((1, n))
    Y = (b @ g) @ Z + b_err @ X_err + Y_err
    prm, prm_err, _, _ = estimate_wtsls(Y, X, Z, w, get_hc1_err)
    dev = prm - b
    summ += np.abs(dev[0][0]) < (prm_err[0].diagonal()[0])**0.5

print(summ / tries)

0.687


In [168]:
n = 300
d = 5
r = 10

Z = rng.standard_normal((r, n))
g = rng.uniform(0, 1, (d, r))
b = rng.uniform(0, 1, (1, d))
b_err = rng.uniform(0, 1, (1, d))
w = rng.uniform(0, 1, n) * 10

X_err = rng.standard_normal((d, n))
X = g @ Z + X_err
Y_err = rng.standard_normal((1, n))
Y = (b @ g) @ Z + b_err @ X_err + Y_err

print_estimate(
    (
        estimate_ols(Y, X, get_hc0_err),
        estimate_ols(Y, X, get_hc1_err),
        estimate_ols(Y, X, get_hc2_err),
        estimate_wls(Y, X, w, get_hc1_err),
        estimate_wls(Y, X, w, get_hc2_err)),
    ('a', 'b', 'c', 'd', 'e'))
print()
print_estimate(
    (
        estimate_tsls(Y, X, Z, get_hc0_err)[:2],
        estimate_tsls(Y, X, Z, get_hc1_err)[:2],
        estimate_tsls(Y, X, Z, get_hc2_err)[:2],
        estimate_wtsls(Y, X, Z, w, get_hc1_err)[:2],
        estimate_wtsls(Y, X, Z, w, get_hc2_err)[:2]),
    ('a', 'b', 'c', 'd', 'e'))
b

 a          b          c          d          e         
 0.637      0.698      0.371      0.84       0.884     
(0.0391  ) (0.0478  ) (0.0439  ) (0.0458  ) (0.0415  ) 
 0.637      0.698      0.371      0.84       0.884     
(0.0394  ) (0.0482  ) (0.0442  ) (0.0461  ) (0.0419  ) 
 0.637      0.698      0.371      0.84       0.884     
(0.0396  ) (0.0483  ) (0.0445  ) (0.0464  ) (0.042   ) 
 0.603      0.675      0.427      0.861      0.887     
(0.0469  ) (0.056   ) (0.0585  ) (0.062   ) (0.0493  ) 
 0.603      0.675      0.427      0.861      0.887     
(0.0476  ) (0.0567  ) (0.0597  ) (0.063   ) (0.0499  ) 

 a          b          c          d          e         
 0.427      0.858      0.565      0.746      0.878     
(0.0563  ) (0.0754  ) (0.0597  ) (0.08    ) (0.0693  ) 
 0.427      0.858      0.565      0.746      0.878     
(0.0623  ) (0.0834  ) (0.066   ) (0.0885  ) (0.0766  ) 
 0.427      0.858      0.565      0.746      0.878     
(0.057   ) (0.0762  ) (0.0604  ) (0.0809  ) (0.

array([[0.44178532, 0.87867674, 0.62825521, 0.70674418, 0.82222899]])

In [185]:
def absorb(Ys, X):
    inv_cov = np.linalg.inv(np.inner(X, X))
    return tuple(Y - (np.inner(Y, X) @ inv_cov) @ X for Y in Ys)

In [64]:
def absorb_fes(Ys, int_fes, real_fes, real_fe_weights, eps=1e-4):
    def absorb_one_int_fe(Ys, fe_sum, fe_inv_ind, fe_counts):
        for Y in Ys:
            for i in range(Y.shape[0]):
                fe_sum[:] = 0
                np.add.at(fe_sum, fe_inv_ind, Y[i])
                fe_sum /= fe_counts
                Y[i] -= fe_sum[fe_inv_ind]

    def absorb_one_real_fe(Ys, fe_sum, fe_inv_ind, fe_cov, fe_weight):
        for Y in Ys:
            for i in range(Y.shape[0]):
                fe_sum[:] = 0
                np.add.at(fe_sum, fe_inv_ind, fe_weight * Y[i])
                fe_sum /= fe_cov
                Y[i] -= fe_sum[fe_inv_ind] * fe_weight

    def absorb_iter(
            Ys, Zs, int_fe_sums, int_fe_inv_inds, int_fe_counts, real_fe_sums,
            real_fe_inv_inds, real_fe_covs, real_fe_weights):
        eps = 0
        for fe_sum, fe_inv_ind, fe_counts in zip(
            int_fe_sums, int_fe_inv_inds, int_fe_counts):
            absorb_one_int_fe(Ys, fe_sum, fe_inv_ind, fe_counts)
        for fe_sum, fe_inv_ind, fe_cov, fe_weight in zip(
            real_fe_sums, real_fe_inv_inds, real_fe_covs, real_fe_weights):
            absorb_one_real_fe(Ys, fe_sum, fe_inv_ind, fe_cov, fe_weight)
        for Z, Y in zip(Zs, Ys):
            eps = np.maximum(eps, np.sqrt(((Z-Y)**2 / Z.size).sum()))
            np.copyto(Z, Y)
        return eps

    int_fe_inv_inds, int_fe_counts = zip(*(
        np.unique(fe, return_inverse=True, return_counts=True)[1:]\
            for fe in int_fes))
    int_fe_sums = tuple(
        np.empty(len(fe_count)) for fe_count in int_fe_counts)
    real_fe_inv_inds = zip(*(
        np.unique(fe, return_inverse=True)[1] for fe in real_fes))
    real_fe_covs = tuple(
        np.empty(fe_inv_ind.max()) for fe_inv_ind in real_fe_inv_inds)
    real_fe_sums = tuple(
        np.empty(len(fe_cov)) for fe_cov in real_fe_covs)
    for fe_inv_ind, fe_weight, fe_cov in zip(
        real_fe_inv_inds, real_fe_weights, real_fe_covs):
        np.add.at(fe_cov, fe_inv_ind, fe_weight**2)

    Zs = tuple(np.empty_like(Y) for Y in Ys)
    for fe_sum, fe_inv_ind, fe_counts in zip(
        int_fe_sums, int_fe_inv_inds, int_fe_counts):
        absorb_one_int_fe(Ys, fe_sum, fe_inv_ind, fe_counts)
        for Z, Y in zip(Zs, Ys):
            np.copyto(Z, Y)
        absorb_one_int_fe(Ys, fe_sum, fe_inv_ind, fe_counts)
        for Z, Y in zip(Zs, Ys):
            eps = np.maximum(eps, np.sqrt(((Z-Y)**2 / Z.size).sum()))
            np.copyto(Z, Y)
    for fe_sum, fe_inv_ind, fe_cov, fe_weight in zip(
        real_fe_sums, real_fe_inv_inds, real_fe_covs, real_fe_weights):
        absorb_one_real_fe(Ys, fe_sum, fe_inv_ind, fe_cov, fe_weight)
        for Z, Y in zip(Zs, Ys):
            np.copyto(Z, Y)
        absorb_one_real_fe(Ys, fe_sum, fe_inv_ind, fe_cov, fe_weight)
        for Z, Y in zip(Zs, Ys):
            eps = np.maximum(eps, np.sqrt(((Z-Y)**2 / Z.size).sum()))
            np.copyto(Z, Y)
    iter_eps = 2 * eps
    iters = 2
    while iter_eps > eps:
        iter_eps = absorb_iter(
            Ys, Zs, int_fe_sums, int_fe_inv_inds, int_fe_counts, real_fe_sums,
            real_fe_inv_inds, real_fe_covs, real_fe_weights)
        iters += 1
    print(iters, eps)
    return Ys

In [65]:
n = 1001
d = 5
group_num = 100

X = rng.standard_normal((d, n))
fe1 = rng.integers(0, 10, n)
fe2 = rng.integers(0, 5, n)
fe3 = rng.integers(0, 7, n)
int_fes = (fe1, fe2, fe3)
real_fes = tuple()
real_fe_weights = tuple()

Y = absorb_fes((X.copy(),), int_fes, real_fes, real_fe_weights)[0]

5 0.0001


In [66]:
absorb_fes((Y.copy(),), int_fes, real_fes, real_fe_weights)

3 0.0001


(array([[ 0.61976706,  0.67286403,  0.01455376, ..., -0.88299497,
          0.11287554,  0.46086876],
        [ 1.32596792, -0.01723872, -2.0954315 , ...,  3.29437931,
         -0.23848466,  0.83593472],
        [-0.98540658, -0.6730087 , -0.58328141, ...,  1.09590977,
          0.29211602, -0.01794524],
        [-2.23957403, -0.59031217,  0.58649004, ..., -0.65545657,
         -0.65380379,  0.04834854],
        [-0.55432767,  0.17049261, -0.92657864, ..., -0.43242741,
         -0.76866692, -1.14397485]]),)

In [67]:
Y

array([[ 0.61976786,  0.67286432,  0.01455404, ..., -0.88299544,
         0.11287669,  0.46086823],
       [ 1.32596916, -0.01723833, -2.09543133, ...,  3.29437887,
        -0.23848368,  0.83593404],
       [-0.98540586, -0.6730085 , -0.58328143, ...,  1.09590962,
         0.29211638, -0.01794555],
       [-2.23957478, -0.59031239,  0.58648975, ..., -0.65545607,
        -0.65380515,  0.04834905],
       [-0.55432788,  0.17049266, -0.92657831, ..., -0.4324277 ,
        -0.76866638, -1.143975  ]])

In [68]:
X

array([[ 0.44786132,  0.61150628, -0.04291387, ..., -0.70656259,
         0.19839476,  0.51654182],
       [ 1.43690275,  0.05921647, -1.98932377, ...,  3.14779131,
        -0.35284057,  1.02808867],
       [-0.94460508, -0.83203732, -0.36515009, ...,  1.11357138,
         0.25777299,  0.21750395],
       [-2.3248494 , -0.85107414,  0.48450332, ..., -0.7746397 ,
        -0.43920506,  0.39139707],
       [-0.53589184,  0.03376505, -0.97536275, ..., -0.44714407,
        -0.69569879, -1.21082812]])